In [61]:
#Imported our dependencies 
#downloaded alot of dependencies 
import os
import json
import pickle
import os.path
import requests
import pandas as pd
from scipy import stats
from config import api_key
import googleapiclient.errors
import matplotlib.pyplot as plt
import googleapiclient.discovery
from __future__ import print_function
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pprint

In [68]:
#Imported and using build() function to build the query url to get our specific data
#used search_cache.json to store a copy of the data on to our system to we dont run out of youtube api quota of the day 
#made it into a dataframe
youtube = build("youtube", "v3", developerKey=api_key)
#snippet_features = ["categoryId", "publishedAt", "channelId", "channelTitle"]
api_run = False 
if not api_run:
    most_viewed = youtube.search().list(part=["snippet", "id"],
                                        type="video", 
                                        publishedAfter="2020-03-15T00:00:00Z", 
                                        publishedBefore="2020-07-15T00:00:00Z",
                                        videoDuration="any", 
                                        order="relevance")
    most_viewed = most_viewed.execute()

print(json.dumps(most_viewed["items"][0],indent=4))
    

{
    "kind": "youtube#searchResult",
    "etag": "RbIgRrbRIdl2h_p0QQf-FQ2uyag",
    "id": {
        "kind": "youtube#video",
        "videoId": "fj28UtF0-Fs"
    },
    "snippet": {
        "publishedAt": "2020-03-25T03:35:31Z",
        "channelId": "UCTkXRDQl0luXxVQrRQvWS6w",
        "title": "Minecraft, But It&#39;s The World Record...",
        "description": "Minecraft, But It's The World Record... My old record was beaten by IlluminaHD, so I had to come back and attempt to take the record back! Follow my socials: ...",
        "thumbnails": {
            "default": {
                "url": "https://i.ytimg.com/vi/fj28UtF0-Fs/default.jpg",
                "width": 120,
                "height": 90
            },
            "medium": {
                "url": "https://i.ytimg.com/vi/fj28UtF0-Fs/mqdefault.jpg",
                "width": 320,
                "height": 180
            },
            "high": {
                "url": "https://i.ytimg.com/vi/fj28UtF0-Fs/hqdefault.jpg",
  

In [70]:
df_mostviewed = pd.DataFrame(most_viewed['items'])
df_mostviewed.to_json('SEARCH_CACHE.json') 
df_mostviewed=pd.DataFrame(pd.read_json('SEARCH_CACHE.json'))      
df_mostviewed

,kind,etag,id,snippet
0,youtube#searchResult,RbIgRrbRIdl2h_p0QQf-FQ2uyag,"{'kind': 'youtube#video', 'videoId': 'fj28UtF0...","{'publishedAt': '2020-03-25T03:35:31Z', 'chann..."
1,youtube#searchResult,_vOx2ehWqgR10WuCdI5CD6cTUs4,"{'kind': 'youtube#video', 'videoId': '5yx6BWlE...","{'publishedAt': '2020-03-21T13:21:02Z', 'chann..."
2,youtube#searchResult,cHsLUYSKMVWLg4DM1MSs7PltE5A,"{'kind': 'youtube#video', 'videoId': '-5KAN9_C...","{'publishedAt': '2020-05-09T14:53:57Z', 'chann..."
3,youtube#searchResult,I9bUa-UDA9OztnaWezl0eUh0gUc,"{'kind': 'youtube#video', 'videoId': 'b9eWlu1Q...","{'publishedAt': '2020-05-26T16:05:21Z', 'chann..."
4,youtube#searchResult,aq_dbr0qNOgAk3u97riSf8yJG44,"{'kind': 'youtube#video', 'videoId': 'HHMvs1ed...","{'publishedAt': '2020-07-14T12:02:11Z', 'chann..."


In [71]:
#normalized the data so we can read/use it easily for analysis
#merged the dataframes together 
df_video_ids = pd.json_normalize(df_mostviewed['id'])
df_snippets = pd.json_normalize(df_mostviewed['snippet']) 
df_merge = pd.merge(df_video_ids , df_snippets, left_index=True, right_index=True)
df_merge.head(1)

,kind,videoId,publishedAt,channelId,title,description,channelTitle,liveBroadcastContent,publishTime,thumbnails.default.url,thumbnails.default.width,thumbnails.default.height,thumbnails.medium.url,thumbnails.medium.width,thumbnails.medium.height,thumbnails.high.url,thumbnails.high.width,thumbnails.high.height
0,youtube#video,fj28UtF0-Fs,2020-03-25T03:35:31Z,UCTkXRDQl0luXxVQrRQvWS6w,"Minecraft, But It&#39;s The World Record...","Minecraft, But It's The World Record... My old...",Dream,none,2020-03-25T03:35:31Z,https://i.ytimg.com/vi/fj28UtF0-Fs/default.jpg,120,90,https://i.ytimg.com/vi/fj28UtF0-Fs/mqdefault.jpg,320,180,https://i.ytimg.com/vi/fj28UtF0-Fs/hqdefault.jpg,480,360


In [72]:
#request another api search by youtube.video().list for the statistics information on the top videos 
#made that into a dataframe
request = youtube.videos().list(part='id, statistics, snippet', id=list(df_top50_merge['videoId']))
response = request.execute()# json
df_stat = pd.DataFrame(response['items']) # response['items'] list of dicts
df_stat.head()

,kind,etag,id,snippet,statistics
0,youtube#video,1FfRf3sdWQW0RJG3oT8oa3FoAb0,iL53Y28Rp84,"{'publishedAt': '2020-03-15T01:45:33Z', 'chann...","{'viewCount': '319761842', 'likeCount': '29662..."
1,youtube#video,pXfNKDpPQzyJgFFtBSrqD24vOJI,NMre6IAAAiU,"{'publishedAt': '2020-04-17T12:57:56Z', 'chann...","{'viewCount': '182377985', 'likeCount': '27418..."
2,youtube#video,oh_TP571bDu4mapsi4f_pIPggKk,tkUvWJiTf9A,"{'publishedAt': '2020-03-19T02:42:21Z', 'chann...","{'viewCount': '133851079', 'likeCount': '17930..."
3,youtube#video,um-m2PWTVDDCYgGUPMgc4uq43mM,d1iwUB9YFnA,"{'publishedAt': '2020-03-24T15:43:55Z', 'chann...","{'viewCount': '114983808', 'likeCount': '12470..."
4,youtube#video,2_XIj0czqYjCNRaWD3WpQ6kJCk4,0k25aI62ENU,"{'publishedAt': '2020-04-15T10:13:41Z', 'chann...","{'viewCount': '83959014', 'likeCount': '22', '..."


In [73]:
#organized the data that was nested in the video id by json.normalize
df_cleanstat = pd.json_normalize(df_stat['statistics']) 
df_cleanstat.head()

,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,319761842,296628,107383,0,0
1,182377985,274182,28163,0,0
2,133851079,179307,33635,0,0
3,114983808,124702,25058,0,0
4,83959014,22,7,0,NaN


In [74]:
#merging the original data and the stats 
df_stat_merge = pd.merge(df_merge, df_cleanstat, left_index=True, right_index=True)
df_stat_merge.head(1)

,kind,videoId,publishedAt,channelId,title,description,channelTitle,liveBroadcastContent,publishTime,thumbnails.default.url,...,thumbnails.medium.width,thumbnails.medium.height,thumbnails.high.url,thumbnails.high.width,thumbnails.high.height,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,youtube#video,fj28UtF0-Fs,2020-03-25T03:35:31Z,UCTkXRDQl0luXxVQrRQvWS6w,"Minecraft, But It&#39;s The World Record...","Minecraft, But It's The World Record... My old...",Dream,none,2020-03-25T03:35:31Z,https://i.ytimg.com/vi/fj28UtF0-Fs/default.jpg,...,320,180,https://i.ytimg.com/vi/fj28UtF0-Fs/hqdefault.jpg,480,360,319761842,296628,107383,0,0


In [75]:
#cleaned up the final dataframe, got rid of the columns that was irrelevant to the project
df_combined = df_stat_merge.drop(columns=['kind','liveBroadcastContent', 'publishTime', 'thumbnails.default.url', 'thumbnails.medium.width', 'thumbnails.default.width', 'thumbnails.high.width', 'thumbnails.high.height', 'thumbnails.default.height', 'thumbnails.medium.url', 'thumbnails.medium.height', 'thumbnails.high.url', 'favoriteCount', 'commentCount'])
df_combined

,videoId,publishedAt,channelId,title,description,channelTitle,viewCount,likeCount,dislikeCount
0,fj28UtF0-Fs,2020-03-25T03:35:31Z,UCTkXRDQl0luXxVQrRQvWS6w,"Minecraft, But It&#39;s The World Record...","Minecraft, But It's The World Record... My old...",Dream,319761842,296628,107383
1,5yx6BWlEVcY,2020-03-21T13:21:02Z,UCOxqgCwgOqC2lMqC5PYz_Dg,Chillhop Radio - jazzy &amp; lofi hip hop beats 🐾,"Welcome to the Chillhop Radio, playing the bes...",Chillhop Music,182377985,274182,28163
2,-5KAN9_CzSA,2020-05-09T14:53:57Z,UCsIg9WMfxjZZvwROleiVsQg,coffee shop radio // 24/7 lofi hip-hop beats,Coffee Shop Vibes on Spotify: https://fanlink....,STEEZYASFUCK,133851079,179307,33635
3,b9eWlu1Qcb0,2020-05-26T16:05:21Z,UCvquBCYgI3nRJWCNCeZ7mgw,Essayez de ne pas rire ( challenge Tiktok ) -...,Steven et moi on se défie à essayez de ne pas ...,The Parodie Bros Show,114983808,124702,25058
4,HHMvs1edihY,2020-07-14T12:02:11Z,UC-gjtyoUTzGEwWM5Y1J4J7Q,Heavy Rain with Thunderstorm | Rain Sounds for...,Heavy rain sounds with heavy thunderstorm 24/7...,New Bliss,83959014,22,7
